# RQT：理論予測 vs 観測値 比較ノートブック（Colab用）
# RQT: Theoretical Predictions vs. Experimental Values (Colab Notebook)
論文 RQT v2.1 の式に基づき、理論予測と観測値を比較する。

Based on the equations presented in the RQT v2.1 paper, this notebook compares theoretical predictions with experimental observations.

In [ ]:
# ✅ 日本語フォント設定（Colab用）
!apt-get -y install fonts-noto-cjk
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
font_path = "/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc"
font_prop = fm.FontProperties(fname=font_path)
plt.rcParams["font.family"] = font_prop.get_name()

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Suggested packages:
  fonts-noto-cjk-extra
The following NEW packages will be installed:
  fonts-noto-cjk
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 61.2 MB of archives.
After this operation, 93.2 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 fonts-noto-cjk all 1:20220127+repack1-1 [61.2 MB]
Fetched 61.2 MB in 3s (22.5 MB/s)
Selecting previously unselected package fonts-noto-cjk.
(Reading database ... 126371 files and directories currently installed.)
Preparing to unpack .../fonts-noto-cjk_1%3a20220127+repack1-1_all.deb ...
Unpacking fonts-noto-cjk (1:20220127+repack1-1) ...
Setting up fonts-noto-cjk (1:20220127+repack1-1) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...


In [ ]:
#RQT のコア式をコード化して、R（世代公比）・PMNS角・CKM主角・ニュートリノ質量（2ケース）を計算する関数群を定義する初期化セル
#Initialization cell that encodes the core formulas of RQT and defines a set of functions to compute R (generation ratios), PMNS angles, CKM primary angles, and neutrino masses (two cases).
import numpy as np, math, pandas as pd, matplotlib.pyplot as plt
deg2rad = np.deg2rad; rad2deg = np.rad2deg
SQRT2 = math.sqrt(2.0)

def Sigma_W(): return 1+1j
def Sigma_EM(): return (1+1j)**2
def Sigma_S(): return (Sigma_EM())**2

def R_unified(dW,Cem,S):
    return (SQRT2**1)*dW + (SQRT2**2)*Cem + (SQRT2**4)*S

def predict_pmns_angles():
    target=1/math.sqrt(2); best=None
    for x_deg in np.linspace(0.1,20,40001):
        x=deg2rad(x_deg); f=math.sin(4*x)+math.sin(x)
        err=abs(f-target)
        if best is None or err<best[0]: best=(err,x_deg)
    th13=best[1]; th12=4*th13; th23=45.0
    return {"theta12":th12,"theta23":th23,"theta13":th13}

def predict_ckm():
    Sd=Sigma_W()+Sigma_EM()+Sigma_S()
    Su=Sigma_W()+Sigma_EM()+2*Sigma_S()
    ratio=abs(Su)/abs(Sd)
    T=(1/(SQRT2**5))*ratio*(1/SQRT2)
    th12=rad2deg(math.atan(T))
    th23=rad2deg(math.atan(T**2))
    th13=rad2deg(math.atan((T**3)/4.0))
    delta=rad2deg(math.atan2(Sigma_W().imag,Sigma_W().real))
    arg_u=math.atan2(Su.imag,Su.real); arg_d=math.atan2(Sd.imag,Sd.real)
    delta+=rad2deg(abs(arg_u-arg_d))
    return {"theta12":th12,"theta23":th23,"theta13":th13,"delta_cp":delta,"T":T}

def nu_masses_case1(dm21,dm31):
    return 0.0, math.sqrt(dm21), math.sqrt(dm31)
def nu_masses_case2(dm21,dm31):
    m1=math.sqrt(dm21); m2=SQRT2*m1; m3=math.sqrt(dm31+m1*m1)
    return m1,m2,m3

In [ ]:
# 観測代表値
# Representative experimental values
R_obs={"neutrino":1.41,"lepton":3.33,"down":7.24,"up":11.10}
PMNS_obs={"theta12":33.7,"theta23":43.3,"theta13":8.6}
CKM_obs={"theta12":13.0,"theta23":2.4,"theta13":0.2,"delta_cp":68.0}
dm21=7.49e-5; dm31=2.513e-3

In [ ]:
# 世代公比R
# Generation ratio R
R_pred={"neutrino":R_unified(1,0,0),"lepton":R_unified(1,1,0),"down":R_unified(1,1,1),"up":R_unified(1,1,2)}
df_R=pd.DataFrame({"family":R_pred.keys(),"R_pred":R_pred.values(),"R_obs":[R_obs[k] for k in R_pred.keys()]})
df_R

,family,R_pred,R_obs
0,neutrino,1.414214,1.41
1,lepton,3.414214,3.33
2,down,7.414214,7.24
3,up,11.414214,11.10


In [ ]:
# PMNS mixing angles
pmns_pred=predict_pmns_angles()
df_pmns=pd.DataFrame({"param":pmns_pred.keys(),"pred_deg":pmns_pred.values(),"obs_deg":[PMNS_obs[k] for k in pmns_pred.keys()]})
df_pmns

,param,pred_deg,obs_deg
0,theta12,34.005130,33.7
1,theta23,45.000000,43.3
2,theta13,8.501283,8.6


In [ ]:
# CKM
ckm_pred=predict_ckm()
df_ckm=pd.DataFrame({"param":["theta12","theta23","theta13","delta_cp"],
                     "pred_deg":[ckm_pred[k] for k in ["theta12","theta23","theta13","delta_cp"]],
                     "obs_deg":[CKM_obs[k] for k in ["theta12","theta23","theta13","delta_cp"]]})
df_ckm

,param,pred_deg,obs_deg
0,theta12,12.646669,13.0
1,theta23,2.882250,2.4
2,theta13,0.161817,0.2
3,delta_cp,66.801409,68.0


In [ ]:
# ニュートリノ質量
# Neutrino masses
m1a,m2a,m3a=nu_masses_case1(dm21,dm31)
m1b,m2b,m3b=nu_masses_case2(dm21,dm31)
pd.DataFrame({"case":["m1=0","Rnu=strict"],"m1":[m1a,m1b],"m2":[m2a,m2b],"m3":[m3a,m3b]})

,case,m1,m2,m3
0,m1=0,0.000000,0.008654,0.050130
1,Rnu=strict,0.008654,0.012239,0.050871
